In [1]:
import reduce

In [2]:
reduce.main()

usage: ipykernel_launcher.py [-h] [-b] [-f] [-o] [-n] [-w] [-s]
                             instrument target
ipykernel_launcher.py: error: the following arguments are required: target


SystemExit: 2

WARNING - /home/tom/venv/PyReduce-mwc4gRME/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)



In [3]:
%tb

SystemExit: 2